In [2]:
import os, shutil, itertools, yaml, kornia, torchvision, sys, copy, math
from functools import partial
import dill as pickle
from importlib import reload
osp = os.path
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
nn = torch.nn
F = nn.functional
import monai.transforms as mtr

In [3]:
from inrnet import args as args_module
from inrnet import jobs as job_mgmt
from inrnet import optim, util, losses
from inrnet.data import dataloader, kitti
from inrnet.experiments import depth
from inrnet import inn
import inrnet.inn.functional as inrF
%matplotlib inline
plt.rcParams["figure.figsize"] = (4.0, 3.0)
plt.rcParams['figure.dpi'] = 200

rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensityRangePercentiles(lower=0, upper=100, b_min=0, b_max=255, clip=False, dtype=np.uint8)
rescale_float = mtr.ScaleIntensity()

TMP_DIR=osp.expanduser("~/code/diffcoord/temp")

rsync -av --ignore-existing \
clintonw@peppercorn.csail.mit.edu:/data/vision/polina/users/clintonw/code/placenta/results/unetr_00 Downloads/

## interactive

In [4]:
args = job_mgmt.get_job_args("inrcoco")
data_loader = dataloader.get_inr_dataloader(args)

In [ ]:
python train.py -j=a6 -c=a6

In [26]:
h,w=8,8
xy_grid = util.meshgrid_coords(h,w)

In [97]:
import torchvision.models as models
m = models.efficientnet_b0()

In [4]:
from inrnet.models.inrs import dvr

In [15]:
D = dvr.Decoder(out_dim=1, c_dim=256)
M = dvr.DVR(D)

In [5]:
sd = torch.load(osp.expanduser("~/code/differentiable_volumetric_rendering/models/ours_depth-64d35726.pt"))

In [29]:
paths = util.glob2("/data/vision/polina/scratch/clintonw/datasets/ShapeNetCore.v2/02691156")

In [56]:
REND_DIR = "/data/vision/polina/scratch/clintonw/datasets/ShapeNetRendering"
imgs = util.glob2(REND_DIR+"/02691156/c2f43dd2c483e0d86b7bd17e458d0dcb/rendering/*.png")

In [40]:
imgs = util.glob2("~/code/differentiable_volumetric_rendering/media/demo/choy_renderings")

In [ ]:
M.load_state_dict(sd["model"], strict=False);

In [27]:
with torch.no_grad():
    out = M.decoder(torch.zeros(1,3))

In [47]:
xy_grids = util.meshgrid_split_coords(4,4)
coords_out = torch.cat(xy_grids, dim=0)
coords_gt = util.meshgrid_coords(4,4)

In [28]:
coords_gt = coords_gt[:,0]*4 + coords_gt[:,1]
coords_out = coords_out[:,0]*4 + coords_out[:,1]

In [39]:
out = coords_out.cpu().numpy().tolist()
indices = [out.index(gt) for gt in coords_gt.cpu().numpy()]

In [75]:
h,w=6,8
xy_grids = util.meshgrid_split_coords(h,w)
coords_out = torch.cat(xy_grids, dim=0)
coords_gt = util.meshgrid_coords(h,w)

In [162]:
N = 6688
gtindices = np.arange(N)
np.random.shuffle(gtindices)
x = torch.randn(N,4)
coords_out = torch.randn(N,2)
coords_gt = coords_out[gtindices]
coords_gt = coords_gt[:,0]*4 + coords_gt[:,1]
coords_out = coords_out[:,0]*4 + coords_out[:,1]

In [192]:
split = 16
N = coords_out.size(0)
dx = N//split
indices = []
for ix in range(0,N,dx):
    matches = coords_gt[ix:ix+dx].unsqueeze(0).cuda() == coords_out.unsqueeze(1).cuda()
    indices.append(torch.where(matches.T)[1])
    if indices[-1].size(0) != dx:
        ok = torch.where(matches.T)[1]
        print(ix, indices[-1].size(0))
        break
indices = torch.cat(indices,dim=0)

1254 419


In [190]:
dx

418

In [188]:
indices.shape

torch.Size([6690])

In [110]:
matches = coords_gt.unsqueeze(1) == coords_out.unsqueeze(0)
indices = torch.where(matches)[1]

In [ ]:
torch.nonzero(coords_gt == coords_out[ix:ix+dx].unsqueeze(0))

In [ ]:
x[indices]

In [106]:
torch.cat(indices,dim=0)

torch.int64

In [80]:
x[indices]

tensor([[-0.6482, -1.3018, -0.9823,  0.5644],
        [ 1.0798,  0.2020, -1.5322, -0.6266],
        [-1.6085, -0.6308, -0.5739, -2.2699],
        [ 1.2115,  0.1361, -0.8018,  0.2340],
        [-0.4681, -0.3251, -0.7570,  0.2249],
        [-1.3283,  2.4344, -2.2429,  0.2720],
        [ 0.2246, -0.5195,  0.2337, -0.3488]])

In [76]:
coco_len = 118287
kitti_len = 80896
horse_len = 1067
zebra_len = 1334

In [ ]:
for ix in range(0,coco_len,1600):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/coco/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"coco{ix//1600}", "fit_coco", ix)

In [ ]:
for ix in range(0,kitti_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"skit{ix//800}", "fit_kitti", ix)

In [ ]:
for ix in range(0,zebra_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/zebra/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"zebra{ix//128}", "fit_zebra", ix)

In [ ]:
for ix in range(0,horse_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/horse/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"horse{ix//128}", "fit_horse", ix)

In [6]:
coords_out.shape

torch.Size([7, 2])

In [ ]:
img = plt.imread("/data/vision/polina/scratch/clintonw/datasets/inrnet/horse2zebra/testA/n02381460_9260.jpg")

In [ ]:
coords_out = coords_out[:,0]*5 + coords_out[:,1]

In [7]:
coords_gt = coords_gt[:,0]*5 + coords_gt[:,1]
coords_out = coords_out[:,0]*5 + coords_out[:,1]

In [14]:
matches = coords_gt.unsqueeze(1) == coords_out.unsqueeze(0)

In [11]:
out

tensor([[-1.7325,  1.4917,  1.0375,  0.0800],
        [ 0.6882, -2.1554, -1.8872,  0.3857],
        [ 1.3248,  2.2584, -0.5773,  1.0302],
        [-1.3867, -2.4084, -0.1861,  0.1831],
        [ 0.6005,  0.0305, -1.4584,  0.5626],
        [ 0.1056,  0.1705,  0.1341, -1.4289],
        [-1.2510,  0.6950,  1.3818,  0.4026]])

In [13]:
coords_gt

tensor([ 4.0818,  5.8486, -7.3113,  2.7945, -4.1563,  2.0609, -3.9877])

In [12]:
coords_out

tensor([ 4.0818, -7.3113, -4.1563, -3.9877,  5.8486,  2.7945,  2.0609])

In [15]:
out[torch.where(matches)[1]]

tensor([[-1.7325,  1.4917,  1.0375,  0.0800],
        [ 0.6005,  0.0305, -1.4584,  0.5626],
        [ 0.6882, -2.1554, -1.8872,  0.3857],
        [ 0.1056,  0.1705,  0.1341, -1.4289],
        [ 1.3248,  2.2584, -0.5773,  1.0302],
        [-1.2510,  0.6950,  1.3818,  0.4026],
        [-1.3867, -2.4084, -0.1861,  0.1831]])

In [ ]:
plt.imshow(img)

In [ ]:
"/data/vision/polina/scratch/clintonw/datasets/coco/annotations/panoptic_train2017"

In [ ]:
osp.exists(f"/data/vision/polina/users/clintonw/code/diffcoord/temp/kitti/siren_{ix+63}.pt")

In [60]:
paths = sorted(util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_*.pt"))

In [ ]:
## search python strings
import os
osp = os.path
root = osp.expanduser("~/code/placenta/placenta")
query_string = 'load_img'
for folder, subfolders, files in os.walk(root):
    for file in files:
        if file.endswith(".py"):
            path = osp.join(folder, file)
            with open(path, 'r') as f:
                if query_string in f.read():
                    print(path)

In [ ]:
coords_gt.shape

In [22]:
out

[4.0818400382995605,
 -7.311278820037842,
 -4.156335830688477,
 -3.98771595954895,
 5.848553657531738,
 2.794525623321533,
 2.060873031616211]